In [1]:
import skimage
import matplotlib.pyplot as plt
from skimage.filters import threshold_otsu
import numpy as np
import os
import scipy.ndimage.morphology
from skimage.measure import label, regionprops
from skimage import feature
from skimage import color
from skimage import filters
import cv2
from sklearn.cluster import KMeans, MiniBatchKMeans
import sklearn
import csv

# Load images

In [2]:
PATH_TO_RESOURCES='../data/images/'
file_paths = !ls $PATH_TO_RESOURCES*.jpg | sort
imgs = [cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY) for path in file_paths]

# Descriptors

In [3]:
desc = []
sift = cv2.xfeatures2d.SIFT_create()

for im in imgs:
    kp, d = sift.detectAndCompute(im, None)
    if d is not None:
        desc.append(d.astype(np.float32))
desc = np.asarray(desc)


In [4]:
train_desc = np.vstack(desc)
print(train_desc.shape)

(48664, 128)


In [5]:
train_mean = np.mean(train_desc, axis=1, keepdims=True)
train_desc = train_desc - train_mean

In [6]:
train_cov = np.dot(train_desc.T, train_desc)
eigvals, eigvecs = np.linalg.eig(train_cov)
perm = eigvals.argsort()                   # sort by increasing eigenvalue
pca_transform = eigvecs[:, perm[64:128]]   # eigenvectors for the 64 last eigenvalues
pca_transform.shape, pca_transform.dtype

((128, 64), dtype('float32'))

In [7]:
train_desc = np.dot(train_desc, pca_transform)

In [8]:
train_desc.shape

(48664, 64)

# K-means

In [9]:
kmeans = MiniBatchKMeans(n_clusters=512, random_state=0).fit(train_desc)

/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)
/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)
/usr/lib/python3.7/site-packages/sklearn/cluster/k_means_.py:1558: RuntimeWarning: init_size=300 should be larger than k=512. Setting it to 3*k
  init_size=init_size)


In [10]:
kmeans.cluster_centers_.shape

(512, 64)

# Split training set

In [35]:
all_images = []
labels = []
with open('../data/gt_img.csv', newline='') as csvfile:
    for line in csvfile:
        line = line.strip()
        im_name, y = line.split(',')
        all_images.append("../data/images/" + im_name + ".jpg")
        labels.append(int(y))
len(all_images), len(labels)

(224, 224)

In [36]:
x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(all_images,
                                                                          labels,
                                                                         test_size=0.2,
                                                                         random_state=42)
len(x_train), len(x_val), len(y_train), len(y_val)

(179, 45, 179, 45)

# Training

In [37]:
l2_normalizer = sklearn.preprocessing.Normalizer(norm='l2', copy=True)
l1_normalizer = sklearn.preprocessing.Normalizer(norm='l1', copy=True)

In [38]:
def compute_descriptors(filelist, train_mean, pca_transform, kmeans):
    image_descriptors = np.zeros((len(filelist), kmeans.n_clusters), dtype=np.float32)
    sift = cv2.xfeatures2d.SIFT_create()
    for ii, file in enumerate(filelist):
        print(file)
        print("Indexing %s" % (file,))
        im = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2GRAY)
        # read the descriptors
        kp, desc = sift.detectAndCompute(im, None)
        if desc is None:
            # let the descriptor be 0 for all values
            # note that this is bad and the element should be dropped from the index
            print("WARNING: zero descriptor for %s" % (file,))
            continue
        
        # convert to float
        desc = desc.astype(np.float32)
        
        # center and apply PCA transform
        desc = desc - np.mean(desc, axis=1, keepdims=True)
        desc = np.dot(desc, pca_transform)
        
        # get cluster ids
        clabels = kmeans.predict(desc)
        # compute histogram
        descr_hist = np.histogram(clabels, bins=kmeans.n_clusters)[0]
        
        descr_hist = np.reshape(descr_hist, (1, -1))
        # l1 norm
        descr_hist = l1_normalizer.transform(descr_hist)
        
        # take the sqrt (Hellinger kernel)
        descr_hist = np.sqrt(descr_hist)
        
        # l2 norm
        descr_hist = l2_normalizer.transform(descr_hist)
        
        # update the index
        image_descriptors[ii] = descr_hist
    print("Indexing complete.")
    return image_descriptors

In [39]:
x_train_desc, x_val_desc = [compute_descriptors(filelist, train_mean, pca_transform, kmeans) 
    for filelist in (x_train, x_val)]

../data/images/img_0073149.jpg
Indexing ../data/images/img_0073149.jpg
../data/images/img_0000068.jpg
Indexing ../data/images/img_0000068.jpg
../data/images/img_0000077.jpg
Indexing ../data/images/img_0000077.jpg
../data/images/img_0073104.jpg
Indexing ../data/images/img_0073104.jpg
../data/images/img_0073140.jpg
Indexing ../data/images/img_0073140.jpg
../data/images/img_0000093.jpg
Indexing ../data/images/img_0000093.jpg
../data/images/img_0073209.jpg
Indexing ../data/images/img_0073209.jpg
../data/images/img_0073102.jpg
Indexing ../data/images/img_0073102.jpg
../data/images/img_0073128.jpg
Indexing ../data/images/img_0073128.jpg
../data/images/img_0000040.jpg
Indexing ../data/images/img_0000040.jpg
../data/images/img_0000025.jpg
Indexing ../data/images/img_0000025.jpg
../data/images/img_0073110.jpg
Indexing ../data/images/img_0073110.jpg
../data/images/img_0073235.jpg
Indexing ../data/images/img_0073235.jpg
../data/images/img_0073251.jpg
Indexing ../data/images/img_0073251.jpg
../dat

../data/images/img_0000086.jpg
Indexing ../data/images/img_0000086.jpg
../data/images/img_0000036.jpg
Indexing ../data/images/img_0000036.jpg
../data/images/img_0000008.jpg
Indexing ../data/images/img_0000008.jpg
../data/images/img_0073126.jpg
Indexing ../data/images/img_0073126.jpg
../data/images/img_0000098.jpg
Indexing ../data/images/img_0000098.jpg
../data/images/img_0000089.jpg
Indexing ../data/images/img_0000089.jpg
../data/images/img_0000279.jpg
Indexing ../data/images/img_0000279.jpg
../data/images/img_0000287.jpg
Indexing ../data/images/img_0000287.jpg
../data/images/img_0000096.jpg
Indexing ../data/images/img_0000096.jpg
../data/images/img_0000009.jpg
Indexing ../data/images/img_0000009.jpg
../data/images/img_0000014.jpg
Indexing ../data/images/img_0000014.jpg
../data/images/img_0000061.jpg
Indexing ../data/images/img_0000061.jpg
../data/images/img_0000278.jpg
Indexing ../data/images/img_0000278.jpg
../data/images/img_0073154.jpg
Indexing ../data/images/img_0073154.jpg
../dat

In [40]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10000,
                              random_state=0)
#clf = svm.SVC()

In [41]:
clf.fit(x_train_desc, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [42]:
clf.score(x_val_desc, y_val)

0.8444444444444444

In [43]:
y_pred = clf.predict(x_val_desc)

In [44]:
from sklearn.metrics import classification_report

print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.90        30
           1       1.00      0.53      0.70        15

    accuracy                           0.84        45
   macro avg       0.91      0.77      0.80        45
weighted avg       0.87      0.84      0.83        45



# Submission

In [45]:
PATH_TO_TEST='../data/imgs_test/'
test_files = !ls $PATH_TO_TEST*.jpg | sort

In [46]:
x_test_desc = compute_descriptors(test_files, train_mean, pca_transform, kmeans)

../data/imgs_test/img_0000100.jpg
Indexing ../data/imgs_test/img_0000100.jpg
../data/imgs_test/img_0000101.jpg
Indexing ../data/imgs_test/img_0000101.jpg
../data/imgs_test/img_0000102.jpg
Indexing ../data/imgs_test/img_0000102.jpg
../data/imgs_test/img_0000103.jpg
Indexing ../data/imgs_test/img_0000103.jpg
../data/imgs_test/img_0000104.jpg
Indexing ../data/imgs_test/img_0000104.jpg
../data/imgs_test/img_0000105.jpg
Indexing ../data/imgs_test/img_0000105.jpg
../data/imgs_test/img_0000107.jpg
Indexing ../data/imgs_test/img_0000107.jpg
../data/imgs_test/img_0000108.jpg
Indexing ../data/imgs_test/img_0000108.jpg
../data/imgs_test/img_0000109.jpg
Indexing ../data/imgs_test/img_0000109.jpg
../data/imgs_test/img_0000110.jpg
Indexing ../data/imgs_test/img_0000110.jpg
../data/imgs_test/img_0000111.jpg
Indexing ../data/imgs_test/img_0000111.jpg
../data/imgs_test/img_0000112.jpg
Indexing ../data/imgs_test/img_0000112.jpg
../data/imgs_test/img_0000113.jpg
Indexing ../data/imgs_test/img_0000113.jpg

In [47]:
y_pred = clf.predict(x_test_desc)

In [55]:
with open("submission.csv", 'w') as sub:
    for i, file in enumerate(test_files):
        image = file.split('/')[3]
        image_name = image.split('.') [0]
        sub.write(image_name + "," + str(y_pred[i]) + "\n")
    #print(file)